### tf-faster-rcnn

--------
This project aim the implement faster rcnn in tensorflow.

In [16]:
from tensorflow.contrib import layers
from tensorflow.contrib.framework.python.ops import arg_scope
from tensorflow.contrib.layers.python.layers import layers as layers_lib
from tensorflow.contrib.layers.python.layers import regularizers
from tensorflow.contrib.layers.python.layers import utils
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import variable_scope
import tensorflow.contrib.slim as slim
import tensorflow as tf
import os

from scipy.misc import imresize

from math import floor,exp
import pprint

import matplotlib.image as mpimg
import numpy as np
import matplotlib.pylab as plt

### Define the config variables 

In [2]:

anchor_box_scales = [128, 256, 512]
anchor_box_ratio = [[1,1],[1,2],[2,1]]


TEST_FULL_IMG = np.array([mpimg.imread("./test1.jpg")])

In [3]:
print(TEST_FULL_IMG.shape)
print(TEST_FULL_IMG.shape[1]/4, TEST_FULL_IMG.shape[2]/4)

(1, 720, 1280, 3)
180.0 320.0


### Define Data generator 

We use VOC 2012 Datasets. I will define funstions to prepare the datasets.

--------
Steps:
1. Read teh annotations files.
2. Create the ground  turth data sets if ground truth does not exist for that image.
3. Define a genertor.


In [17]:
# import lib
import os
import cv2
import xml.etree.ElementTree as ET
import numpy as np

#### Define funciton for get the data from the annotations 

In [22]:
dataSets = ['VOC2012']


def get_vox_data(input_path):
    all_imgs = []

    classes_count = {}

    class_mappingNameToId = {}
    class_mappingIdToName = {}

    visualise = False

    data_paths = [os.path.join(input_path, s) for s in dataSets]

    print ('Parsing annotation files')

    for data_path in data_paths:

        annot_path = os.path.join(data_path, 'Annotations')
        imgs_path = os.path.join(data_path, 'JPEGImages')
        imgsets_path_trainval = os.path.join(data_path, 'ImageSets',
                'Main', 'trainval.txt')

# ........imgsets_path_test = os.path.join(data_path, 'ImageSets','Main','test.txt')

        trainval_files = []
        test_files = []
        try:
            with open(imgsets_path_trainval) as f:
                for line in f:
                    trainval_files.append(line.strip() + '.jpg')
        except Exception as e:

# ............with open(imgsets_path_test) as f:
# ................for line in f:
# ....................test_files.append(line.strip() + '.jpg')

            print (e)

        annots = [os.path.join(annot_path, s) for s in
                  os.listdir(annot_path)]
        idx = 0
        for annot in annots:
            try:
                idx += 1

                et = ET.parse(annot)
                element = et.getroot()

                element_objs = element.findall('object')
                element_filename = element.find('filename').text
                element_width = int(element.find('size').find('width'
                                    ).text)
                element_height = int(element.find('size').find('height'
                        ).text)

                if len(element_objs) > 0:
                    annotation_data = {
                        'filepath': os.path.join(imgs_path,
                                element_filename),
                        'width': element_width,
                        'height': element_height,
                        'bboxes': [],
                        }

                    if element_filename in trainval_files:
                        annotation_data['imageset'] = 'trainval'
                    elif element_filename in test_files:
                        annotation_data['imageset'] = 'test'
                    else:
                        annotation_data['imageset'] = 'trainval'

                for element_obj in element_objs:
                    class_name = element_obj.find('name').text
                    if class_name not in classes_count:
                        classes_count[class_name] = 1
                    else:
                        classes_count[class_name] += 1

                    if class_name not in class_mappingNameToId:
                        class_mappingNameToId[class_name] = \
                            len(class_mappingNameToId)
                        class_mappingIdToName[len(class_mappingNameToId)
                                - 1] = class_name

                    obj_bbox = element_obj.find('bndbox')
                    x1 = int(round(float(obj_bbox.find('xmin').text)))
                    y1 = int(round(float(obj_bbox.find('ymin').text)))
                    x2 = int(round(float(obj_bbox.find('xmax').text)))
                    y2 = int(round(float(obj_bbox.find('ymax').text)))
                    w = x2 - x1
                    h = y2 - y1
                    x = int(round(x1 + w / 2))
                    y = int(round(y1 + h / 2))
                    difficulty = int(element_obj.find('difficult'
                            ).text) == 1
                    annotation_data['bboxes'].append({
                        'class': class_name,
                        'x1': x1,
                        'x2': x2,
                        'y1': y1,
                        'y2': y2,
                        'difficult': difficulty,
                        'x': x,
                        'y': y,
                        'w': w,
                        'h': h,
                        })
                all_imgs.append(annotation_data)

                if visualise:
                    img = cv2.imread(annotation_data['filepath'])
                    for bbox in annotation_data['bboxes']:
                        cv2.rectangle(img, (bbox['x1'], bbox['y1']),
                                (bbox['x2'], bbox['y2']), (0, 0, 255))
                    cv2.imshow('img', img)
                    cv2.waitKey(0)
            except Exception as  e:

                print (e)
                continue

        # make if no bg in the className make bg class

        if 'bg' not in classes_count:
            print ('bg not in class')
            classes_count['bg'] = 0
            class_mappingNameToId['bg'] = len(class_mappingNameToId)
            class_mappingIdToName[len(class_mappingNameToId)] = 'bg'
        else:
            print ('bg in class')

        # if there are a bg class make it last index

            tempId = class_mappingNameToId['bg']
            class_mappingNameToId['bg'] = len(class_mappingNameToId) - 1
            lastName = class_mappingIdToName[len(class_mappingNameToId)
                    - 1]
            class_mappingNameToId[lastName] = tempId
            class_mappingIdToName[tempId] = lastName
            class_mappingIdToName[len(class_mappingNameToId)] = 'bg'
            
    return (all_imgs, classes_count, class_mappingNameToId, class_mappingIdToName)

#### Define the function for generating ground truth 
--------------


In [ ]:
def generate_ground_truth():
    

In [24]:
all_imgs, classes_count, class_mappingNameToId, class_mappingIdToName = get_vox_data("./data")

Parsing annotation files
bg not in class


### Define network 
Vgg 16 used in this project, it use the pretrain network from imageNet. The network defined until conv5.

The vgg16 function accept iamge input and otput the nets tensor and the enpoints.

We use conv5 as our feature map layer, which will port to rpn and rcnn.

In [4]:
# Define the vgg16 layers
def vgg_16(inputs,  scope='vgg_16'):
    with tf.variable_scope(scope, 'vgg_16', [inputs]) as sc:
        end_points_collection = sc.name + '_end_points'

        # Collect outputs for conv2d, fully_connected and max_pool2d.
        with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d], outputs_collections=end_points_collection):
            net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
            net = slim.max_pool2d(net, [2, 2], scope='pool1')
            net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
            net = slim.max_pool2d(net, [2, 2], scope='pool2')
            net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3')
#             net = slim.max_pool2d(net, [2, 2], scope='pool3')
            net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv4')
#             net = slim.max_pool2d(net, [2, 2], scope='pool4')
            net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv5')

            # Convert end_points_collection into a end_point dict.
            end_points = slim.utils.convert_collection_to_dict(end_points_collection)
        
    return net, end_points


### Define RPN netowrk

we will define the RPN network. 

It have few steps. 
1. Define the conv layers of rpn.
2. Mapping rpn to bbox
3. Poposal layer, filter the bbox that needed 

In [ ]:
def rpn(net, num_anchors=9, scope="rpn"):
    with tf.variable_scope(scope, 'rpn', [net]) as sc:
        end_points_collection = sc.name + '_end_points'
        
        with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d], 
                            outputs_collections=end_points_collection, 
                            activation_fn=tf.nn.relu,
                            weights_initializer=tf.truncated_normal_initializer(0.0, 0.01)):
            
            net = slim.conv2d(net, 512, [3, 1], scope='rpn_conv_3x3', padding='SAME')
            
            rpn_class = slim.conv2d(net, num_anchors, [1, 1], scope='rpn_class')
            
            
            rpn_regr = slim.conv2d(net, num_anchors*4, [1, 1], scope='rpn_regr')   

            # Convert end_points_collection into a end_point dict.
            end_points = slim.utils.convert_collection_to_dict(end_points_collection)
        
    return rpn_class, rpn_regr, end_points

#### Define mapping from network result to bboxes

In [ ]:

# TODO use transpose to speed up https://www.tensorflow.org/api_docs/python/tf/transpose
def mapAnchorToBoxs(rpn_class, rpn_regr, feature_map_ratio, ratio=[(1,1),(2,1),(1,2)], pixel=[64,128,512]):
    
    class_res  = [];
    regr_res = [];
    
    
    for row in range(rpn_class.shape[0]):
        for col in range(rpn_class.shape[1]):
            i = 0
            for r in ratio:
                j = 0
                for p in pixel:
                    anchor_width = r[0]*p / feature_map_ratio
                    anchor_height = r[1]*p / feature_map_ratio
                    
                    p_anchor = rpn_regr[row][col][i+j]
                    
                    cx = row + p_anchor[0]*anchor_width
                    cy = col + p_anchor[1]*anchor_height
                    pred_w = exp(p_anchor[2]) * anchor_width
                    pred_h = exp(p_anchor[3]) * anchor_height
                    
                    x1 = int(cx - 0.5 * pred_w)
                    y1 = int(cy - 0.5 * pred_h)
                    x2 = int(cx + 0.5 * pred_w)
                    y2 = int(cy + 0.5 * pred_h)
                    
                    regr_res.append([x1,y1,x2,y2])
                    class_res.append(rpn_class[row][col][i+j])
#                     print("rpn_class[row][col][i+j]",rpn_class[row][col][i+j])
                    
                    
                    j+=1
                i+=1
    
    return np.array(class_res), np.array(regr_res)


# def bbox_transform_inv(boxes, deltas):
#     if boxes.shape[0] == 0:
#         return np.zeros((0, deltas.shape[1]), dtype=deltas.dtype)

#     boxes = boxes.astype(deltas.dtype, copy=False)

#     widths = boxes[:, 2] - boxes[:, 0] + 1.0
#     heights = boxes[:, 3] - boxes[:, 1] + 1.0
#     ctr_x = boxes[:, 0] + 0.5 * widths
#     ctr_y = boxes[:, 1] + 0.5 * heights

#     dx = deltas[:, 0::4]
#     dy = deltas[:, 1::4]
#     dw = deltas[:, 2::4]
#     dh = deltas[:, 3::4]

#     pred_ctr_x = dx * widths[:, np.newaxis] + ctr_x[:, np.newaxis]
#     pred_ctr_y = dy * heights[:, np.newaxis] + ctr_y[:, np.newaxis]
#     pred_w = np.exp(dw) * widths[:, np.newaxis]
#     pred_h = np.exp(dh) * heights[:, np.newaxis]

#     pred_boxes = np.zeros(deltas.shape, dtype=deltas.dtype)
#     # x1
#     pred_boxes[:, 0::4] = pred_ctr_x - 0.5 * pred_w
#     # y1
#     pred_boxes[:, 1::4] = pred_ctr_y - 0.5 * pred_h
#     # x2
#     pred_boxes[:, 2::4] = pred_ctr_x + 0.5 * pred_w
#     # y2
#     pred_boxes[:, 3::4] = pred_ctr_y + 0.5 * pred_h

#     return pred_boxes


def getLargest(a,b):
    if (a>b):
        return a
    else :
        return b

def getSmallest(a,b):
    if(a<b):
        return a
    else:
        return b
    
    
def clip_boxes(boxes, im_shape_col, im_shape_row):
    """
    Clip boxes to image boundaries.
    """
    for box in boxes:
        box[0] = getLargest( getSmallest(box[0], im_shape_col) ,0)
        box[1] = getLargest( getSmallest(box[1], im_shape_row) ,0)
        box[2] = getLargest( getSmallest(box[2], im_shape_col) ,0)
        box[3] = getLargest( getSmallest(box[3], im_shape_row) ,0)
            
    return boxes

def filter_boxs_by_size(boxes, threshold=30):
    
    ws = boxes[:, 2] - boxes[:, 0] + 1
    hs = boxes[:, 3] - boxes[:, 1] + 1
    keep = np.where((ws >= threshold) & (hs >= threshold))[0]
    return keep

def nms(dets, thresh):
    """Pure Python NMS baseline."""
    x1 = dets[:, 0]
    y1 = dets[:, 1]
    x2 = dets[:, 2]
    y2 = dets[:, 3]
    scores = dets[:, 4]

    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        inds = np.where(ovr <= thresh)[0]
        order = order[inds + 1]

    return keep

def rpn_proposal_layer(rpn_class, rpn_regr, img_input):
    ''' 
    This function map anchor to the position, which return x,y,w,h with globel coor. 
    rather than anchor inner position.
    img_input is use for get the imgSize ref.
    '''
    
    print("rpn_class.shape", rpn_class.shape)
    print("rpn_regr.shape", rpn_regr.shape)
    print("img_input.shape", img_input.shape)
    
    img_width = img_input.shape[2]
    img_height = img_input.shape[1]
    
    feature_map_width = rpn_regr.shape[2]
    feature_map_height = rpn_regr.shape[1]
    
    img_to_feature_map_ratio = img_width/feature_map_width
    
    nb_anchors = int(rpn_class.shape[3])
    
#     reshape the array by anchors 
    rpn_class_reshaped = np.reshape(rpn_class, (int(rpn_class.shape[1]), int(rpn_class.shape[2]),nb_anchors, 1))
    rpn_regr_reshaped = np.reshape(rpn_regr, (int(rpn_regr.shape[1]), int(rpn_regr.shape[2]),nb_anchors, 4)) 
    
    class_res, regr_res = mapAnchorToBoxs(rpn_class_reshaped, rpn_regr_reshaped, img_to_feature_map_ratio)
    
    # 2. clip predicted boxes to image
    regr_res = clip_boxes(regr_res, int(rpn_class.shape[2]), int(rpn_class.shape[1]))
    print("class_res.shape",class_res.shape)
    print("regr_res.shape",regr_res.shape)
    
    # filter by size
    id_keep = filter_boxs_by_size(regr_res, 30)
    print("id_keep.shape",id_keep.shape)
    class_res_keep = class_res[id_keep]
    regr_res_keep = regr_res[id_keep, :]
    
    # sort 
    # getTop N 
    pre_nms_topN = 6000
    
    order = class_res_keep.ravel().argsort()[::-1]
    if pre_nms_topN > 0:
        order = order[:pre_nms_topN]
    proposals = regr_res_keep[order, :]
    scores = class_res_keep[order]
    
    print("proposals.shape",proposals.shape)    
    print("scores.shape",scores.shape)
    
    # 6. apply nms (e.g. threshold = 0.7)
    # 7. take after_nms_topN (e.g. 300)
    # 8. return the top proposals (-> RoIs top)
    post_nms_topN = 300
    nms_thresh = 0.6
    keep = nms(np.hstack((proposals, scores)), nms_thresh)
    if post_nms_topN > 0:
        keep = keep[:post_nms_topN]
    proposals = proposals[keep, :]
    scores = scores[keep]
    print("proposals.shape",proposals.shape)    
    print("scores.shape",scores.shape)
    
    return proposals.astype(np.float32) 

In [5]:
def getBoxIds(poposal_res):
    print("poposal_res.shape",poposal_res.shape)
    return np.zeros(len(poposal_res)).astype(np.int32)

### Define loss

-------------
#### Training RPNs 
I fllow the method in paper, which random sample 256 anchors in image and make pos. neg anchor be 1:1 ratio. 
#### Method to assign pos. label to anchor
1. the anchor with highest intersection-over-Union (IoU) overla[ with a geound truth box or
2. an anchor that has an IoU overlap with higher than 0.7 withany ground truth box 

#### Method to assign neg. label to anchor 
1. IoU ratio is lower than 0.3 for all ground truth boxes

if anchor neither pos. and neg. do not contribute to the training objective 

In [ ]:
def smooth_l1(self, sigma, bbox_pred, bbox_targets, bbox_inside_weights, bbox_outside_weights):
    """
        ResultLoss = outside_weights * SmoothL1(inside_weights * (bbox_pred - bbox_targets))
        SmoothL1(x) = 0.5 * (sigma * x)^2,    if |x| < 1 / sigma^2
                      |x| - 0.5 / sigma^2,    otherwise
    """
    sigma2 = sigma * sigma

    x = tf.subtract(bbox_pred, bbox_targets)

    smooth_l1_sign = tf.cast(tf.less(tf.abs(x), 1.0 / sigma2), tf.float32)
    smooth_l1_option1 = tf.multiply(tf.multiply(x, x), 0.5 * sigma2)
    smooth_l1_option2 = tf.subtract(tf.abs(x), 0.5 / sigma2)
    smooth_l1_result = tf.add(tf.multiply(smooth_l1_option1, smooth_l1_sign),
                              tf.multiply(smooth_l1_option2, tf.abs(tf.subtract(smooth_l1_sign, 1.0))))

    return smooth_l1_result

In [ ]:
lambda_rpn_regr = 1.0
lambda_rcnn_regr = 1.0
lambda_rpn_class = 1.0
lambda_rcnn_class = 1.0

esilon = 1e-4


def rpn_regr_loss(y_pred, y_true_regr, y_true_cls):
    x = y_pred - y_true_regr 
    x_abs = tf.abs(x)
    x_bool = tf.cast(tf.less_equal(x_abs, 1.0), tf.float32)
    
    x_smooth_l1 = (x_bool * (0.5 * x * x) + (1 - x_bool) * (x_abs - 0.5))
    x_smooth_l1_pos_only = y_true_cls * x_smooth_l1
    
    return lambda_rpn_regr * x_smooth_l1_pos_only / tf.sum(epsilon + y_true_cls)
    
def rpn_class_loss(y_pred, y_true_cls):
    x = y_true_cls * tf.binary_crossentropy(y_pred, y_true_cls)
    return lambda_rpn_class * tf.sum(x) / tf.sum(epsilon + y_true_cls)

# background is id = 0
def rcnn_regr_loss(y_pred, y_true_regr, y_true_cls):
    x = y_pred - y_true_regr
    x_abs = tf.abs(x)
    x_bool = tf.cast(tf.less_equal(x_abs, 1.0), tf.float32)
    
    x_smooth_l1 = (x_bool * (0.5 * x * x) + (1 - x_bool) * (x_abs - 0.5))
    x_smooth_l1_pos_only = y_true_cls * x_smooth_l1
    
    return lambda_rpn_regr * x_smooth_l1_pos_only / tf.sum(epsilon + y_true_cls)
    

def rcnn_cls_loss(y_pred, y_true):
    softmax_cross_entropy = tf.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    return lambda_rcnn_class * tf.reduce_mean(softmax_cross_entropy)

In [ ]:
def rpn_ground_truth(x_image, y_bbox_input):
    '''
        1. get the feature map size 
        2. generate two matrix ground truth, cls ground truth base on bbox 
    '''
    pass

def rpn_y_bbox(rpn_ground_truth_res):
    return rpn_ground_truth_res[0]

def rpn_y_cls(rpn_ground_truth_res):
    return rpn_ground_truth_res[1]

### Build network

In [14]:
nb_anchors = len(anchor_box_scales) * len(anchor_box_ratio)

img_input = tf.placeholder(tf.float32, [1, None, None, 3])

# input the bbox location and the class id 
# in format x,y,w,h,classId
y_bbox_input = tf.placeholder(tf.float32, [1, None, 5])

# generate rpn ground truth 
rpn_y_res = tf.py_func(rpn_ground_truth, [img_input, y_bbox_input], tf.float32, name="rpn_y_res")
rpn_y_bbox_res = tf.py_func(rpn_y_bbox,rpn_y_res, tf.float32, name="rpn_y_bbox_res")
rpn_y_cls_res = tf.py_func(rpn_y_cls,rpn_y_res, tf.float32, name="rpn_y_cls_res")

# define the crop size of roi pooling 
crop_size = tf.constant([14,14])

conv_layer,conv_end_points = vgg_16(img_input)
conv_restore_names = [ item for item in conv_end_points] 

rpn_class, rpn_regr, rpn_end_points = rpn(conv_layer,nb_anchors)
rpn_class_softmax = tf.nn.softmax(rpn_class)

# poposal layer
roi_proposal = tf.py_func(rpn_proposal_layer,[rpn_class_softmax, rpn_regr, img_input], tf.float32, name="roi_proposal")

rpn_loss_regr = rpn_regr_loss(rpn_regr, rpn_y_bbox_res)
rpn_loss_cls = rpn_class_loss(rpn_class, rpn_y_cls_res)
rpn_loss = rpn_loss_regr + rpn_loss_cls

rpn_optimizer = tf.train.AdamOptimizer(0.005).minimize(rpn_loss)

# pooling layer
get_box_ids = tf.py_func(getBoxIds, [roi_proposal],  tf.int32 );

roi_pooling = tf.image.crop_and_resize(conv_layer,roi_proposal, get_box_ids, crop_size)
roi_pooling = tf.nn.max_pool(roi_pooling, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],  padding='SAME')

flatten = slim.flatten(roi_pooling, scope="flatten")
fc6 = slim.fully_connected(flatten, 4096)
fc7 = slim.fully_connected(fc6, 4096)

cls_score = slim.fully_connected(fc7, 21)  # 20 class + bg 
cls_score = tf.nn.softmax(cls_score)

bbox_pred = slim.fully_connected(fc7, 84) # each class have it's own bbox



# rcnn_loss_regr = rcnn_regr_loss()
# rcnn_loss_cls = rcnn_cls_loss()
# rcnn_loss = rcnn_loss_regr + rcnn_loss_cls
# rpn_optimizer = tf.train.AdamOptimizer(0.005).minimize(rcnn_loss)


# restore weights
variables_to_restore = slim.get_variables_to_restore(include=conv_restore_names)
vgg_checkpoint_path = os.path.join("./", 'vgg_16.ckpt')
restorer = tf.train.Saver(variables_to_restore)


init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    # Restore variables from disk.
    restorer.restore(sess, "./vgg_16.ckpt")
    
    sess.run(init_op)
    print("restore conv layers")
    
    res, rpn_class_res, rpn_regr_res, roi_proposal_res, roi_pooling_res, bbox_pred_res, cls_score_res = sess.run([conv_layer, rpn_class, rpn_regr, roi_proposal, roi_pooling, bbox_pred, cls_score], feed_dict={img_input:TEST_FULL_IMG})
#     plt.imshow(res[0], cmap='gray')
    

restore conv layers
rpn_class.shape (1, 180, 320, 9)
rpn_regr.shape (1, 180, 320, 36)
img_input.shape (1, 720, 1280, 3)
class_res.shape (518400, 1)
regr_res.shape (518400, 4)
id_keep.shape (211606,)
proposals.shape (6000, 4)
scores.shape (6000, 1)
proposals.shape (85, 4)
scores.shape (85, 1)
poposal_res.shape (85, 4)


In [15]:
print(" res.shape",res.shape, "\n rpn_class_res.shape",rpn_class_res.shape, "\n rpn_regr_res.shape", rpn_regr_res.shape, "\n roi_proposal_res.shape", roi_proposal_res.shape, "\n roi_pooling_res.shape", roi_pooling_res.shape, "\n bbox_pred.shape",bbox_pred_res.shape, "cls_score.shape",cls_score_res.shape)

 res.shape (1, 180, 320, 512) 
 rpn_class_res.shape (1, 180, 320, 9) 
 rpn_regr_res.shape (1, 180, 320, 36) 
 roi_proposal_res.shape (85, 4) 
 roi_pooling_res.shape (85, 7, 7, 512) 
 bbox_pred.shape (85, 84) cls_score.shape (85, 21)


### Training